# Instancing Spark

Import the PySpark library and the `SparkSession` class

In [92]:
import pyspark
from pyspark.sql import SparkSession

Instance a session

In [97]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test1') \
    .config("spark.driver.port", "4043") \
    .getOrCreate()

ConnectionRefusedError: [Errno 61] Connection refused

# Read data

Download some large data

In [49]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

--2023-06-21 03:28:41--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 54.231.228.201, 52.217.124.249, 54.231.133.169, ...
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|54.231.228.201|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2023-06-21 03:28:41 ERROR 403: Forbidden.



Read the data. Note that the datatypes are not inferred; everything is a string.

In [50]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [51]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [52]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

Read the schema

In [53]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

# Creating a schema with Pandas

Write the first 1001 lines of the CSV file to another file

In [54]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

Count the lines in the new CSV file

In [55]:
!wc -l head.csv

    1001 head.csv


We will now use Pandas to infer the datatypes. Let's import Pandas and read the small CSV file.

In [56]:
import pandas as pd

In [57]:
df_pandas = pd.read_csv('head.csv')

Let's see the inferred types.

In [58]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

Spark can create dataframes from Pandas dataframes. Let's see how Spark converts it and check the datatypes as defined in Spark.

In [59]:
spark.createDataFrame(df_pandas).show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|    NaN|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|    NaN|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|    NaN|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|    NaN|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|    NaN|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [60]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

This datatype description is from Scala, which is the language that Spark is created with. Let's clean up the output and create our usable schema from it.

In [61]:
# Don't run this block!
'''
StructType(
    List(
        StructField(hvfhs_license_num,StringType,true),
        StructField(dispatching_base_num,StringType,true),
        StructField(pickup_datetime,StringType,true),
        StructField(dropoff_datetime,StringType,true),
        StructField(PULocationID,LongType,true),
        StructField(DOLocationID,LongType,true),
        StructField(SR_Flag,DoubleType,true)
    )
)
'''

'\nStructType(\n    List(\n        StructField(hvfhs_license_num,StringType,true),\n        StructField(dispatching_base_num,StringType,true),\n        StructField(pickup_datetime,StringType,true),\n        StructField(dropoff_datetime,StringType,true),\n        StructField(PULocationID,LongType,true),\n        StructField(DOLocationID,LongType,true),\n        StructField(SR_Flag,DoubleType,true)\n    )\n)\n'

The timestamps were not originally inferred by Pandas but we know which fields should be datatypes. We will convert longs (`int64`) to integers for memory optimization, and convert `SR_Flag` to string.

In [62]:
from pyspark.sql import types

In [63]:
schema = types.StructType(
    [
        types.StructField('hvfhs_license_num', types.StringType(), True),
        types.StructField('dispatching_base_num', types.StringType(), True),
        types.StructField('pickup_datetime', types.TimestampType(), True),
        types.StructField('dropoff_datetime', types.TimestampType(), True),
        types.StructField('PULocationID', types.IntegerType(), True),
        types.StructField('DOLocationID', types.IntegerType(), True),
        types.StructField('SR_Flag', types.StringType(), True)
    ]
)

With this new schema, we can now create a dataframe with inferred datatypes.

In [64]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [65]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [66]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

# Partitions

We will now create 24 partitions in our dataframe

In [67]:
df = df.repartition(24)

Let's parquetize the dataframe. This will create 24 smaller parquet files.

This operation may take a while.

In [68]:
df.write.parquet('fhvhv/2021/01/', mode='overwrite')

Check the created files

In [69]:
!ls -lh fhvhv/2021/01/

total 445440
-rw-r--r--  1 chitra  staff     0B Jun 21 03:29 _SUCCESS
-rw-r--r--  1 chitra  staff   9.0M Jun 21 03:29 part-00000-72ffb0ae-4d19-4ca1-899b-3a02a6026ebf-c000.snappy.parquet
-rw-r--r--  1 chitra  staff   9.0M Jun 21 03:29 part-00001-72ffb0ae-4d19-4ca1-899b-3a02a6026ebf-c000.snappy.parquet
-rw-r--r--  1 chitra  staff   8.9M Jun 21 03:29 part-00002-72ffb0ae-4d19-4ca1-899b-3a02a6026ebf-c000.snappy.parquet
-rw-r--r--  1 chitra  staff   9.0M Jun 21 03:29 part-00003-72ffb0ae-4d19-4ca1-899b-3a02a6026ebf-c000.snappy.parquet
-rw-r--r--  1 chitra  staff   9.0M Jun 21 03:29 part-00004-72ffb0ae-4d19-4ca1-899b-3a02a6026ebf-c000.snappy.parquet
-rw-r--r--  1 chitra  staff   9.0M Jun 21 03:29 part-00005-72ffb0ae-4d19-4ca1-899b-3a02a6026ebf-c000.snappy.parquet
-rw-r--r--  1 chitra  staff   8.9M Jun 21 03:29 part-00006-72ffb0ae-4d19-4ca1-899b-3a02a6026ebf-c000.snappy.parquet
-rw-r--r--  1 chitra  staff   8.9M Jun 21 03:29 part-00007-72ffb0ae-4d19-4ca1-899b-3a02a6026ebf-c000.snappy.parquet
-r

# Spark Dataframes

Create a dataframe from the parquet files.

In [70]:
df = spark.read.parquet('fhvhv/2021/01/')

Check the schema

In [71]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



# Functions and UDFs

In [72]:
from pyspark.sql import functions as F

In [73]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02877|2021-01-04 08:38:43|2021-01-04 08:55:29|         235|          32|   null|
|           HV0003|              B02875|2021-01-05 06:07:51|2021-01-05 06:21:03|         145|         202|   null|
|           HV0003|              B02765|2021-01-01 19:31:34|2021-01-01 20:31:12|         185|         251|   null|
|           HV0005|              B02510|2021-01-01 00:36:14|2021-01-01 01:05:08|          47|         146|   null|
|           HV0003|              B02764|2021-01-03 00:28:38|2021-01-03 00:33:57|         248|         208|   null|
|           HV0003|              B02876|2021-01-01 00:07:39|2021-01-01 00:24:33|

Using built-in functions (`F.to_date()`)

In [74]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
| 2021-01-04|  2021-01-04|         235|          32|
| 2021-01-05|  2021-01-05|         145|         202|
| 2021-01-01|  2021-01-01|         185|         251|
| 2021-01-01|  2021-01-01|          47|         146|
| 2021-01-03|  2021-01-03|         248|         208|
| 2021-01-01|  2021-01-01|          17|         148|
| 2021-01-02|  2021-01-02|          74|         247|
| 2021-01-01|  2021-01-01|         169|         167|
| 2021-01-05|  2021-01-05|         219|         180|
| 2021-01-06|  2021-01-06|         112|         145|
| 2021-01-03|  2021-01-03|          39|          35|
| 2021-01-04|  2021-01-04|          69|         262|
| 2021-01-04|  2021-01-04|         221|         221|
| 2021-01-06|  2021-01-06|         124|          89|
| 2021-01-06|  2021-01-06|         125|         161|
| 2021-01-06|  2021-01-06|          61|       

Creating a UDF

In [75]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [76]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  s/b3d| 2021-01-04|  2021-01-04|         235|          32|
|  e/b3b| 2021-01-05|  2021-01-05|         145|         202|
|  s/acd| 2021-01-01|  2021-01-01|         185|         251|
|  e/9ce| 2021-01-01|  2021-01-01|          47|         146|
|  e/acc| 2021-01-03|  2021-01-03|         248|         208|
|  e/b3c| 2021-01-01|  2021-01-01|          17|         148|
|  e/9ce| 2021-01-02|  2021-01-02|          74|         247|
|  e/b38| 2021-01-01|  2021-01-01|         169|         167|
|  a/b43| 2021-01-05|  2021-01-05|         219|         180|
|  e/9ce| 2021-01-06|  2021-01-06|         112|         145|
|  s/acd| 2021-01-03|  2021-01-03|          39|          35|
|  e/b30| 2021-01-04|  2021-01-04|          69|         262|
|  e/b32| 2021-01-04|  2021-01-04|         221|         221|
|  e/b38| 2021-01-06|  2

23/06/21 10:06:49 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 19369861 ms exceeds timeout 120000 ms
23/06/21 10:06:50 WARN SparkContext: Killing executors is not supported by current scheduler.
23/06/21 10:06:51 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:641)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1111)
	at org.apache.spark.executor.Executor.$anonfun$heartbeater$1(Executor.scala:244)
	at 

In [96]:
spark.stop()

ConnectionRefusedError: [Errno 61] Connection refused